## Main message
Derivatives of a scalar with respect to a scalar might be relatively straightforward. Derivatives of vector valued functions are not impossibly difficult. You can use intelligent matrix and array operations to facilitate the process.

## What are vector valued functions?
A vector valued function is any function that returns multiple values (outputs). These are quite common in engineering models; rarely do we only care about scalar inputs and scalar outputs to a model. Here are some examples of vector valued functions in the wild:
- structural stresses in different members in an aircraft wing
- thickness of the tower for a wind turbine from base to top
- thrust outputs of an engine at different operating points
- altitude of an aircraft along different points in its trajectory

Intuitively, each of these quantities should be considered together, so it makes sense to contain them within a vector or array. You *could* consider each of them as scalar values, but it would be unreasonably difficult to track them through your model and perform computations efficiently.

In a general sense, think of a "vector" as including arbitrarily dimensioned arrays and tensors. Strictly mathematically, "vector valued functions" only refer to functions that produce multidimensional outputs, but this lesson is relevant even in the case of multidimensional inputs and single-dimensional outputs. I'll be covering the general case of arbitrarily sized inputs and outputs.

Here are some three examples of engineering functions that are not scalar-to-scalar.


### Angle between two 3D vectors
Despite having 6 inputs, because the output is scalar, this is technically not a vector valued function. It's still quite a relevant engineering function. Don't read too much into the proper definition of "vector valued function" and what's included or not, just know that in this lesson we're talking about anything that's not a scalar-to-scalar function.

$$
\begin{align*}
  {\bf a} &= \begin{bmatrix}
          a_{x} \\
          a_{y} \\
          a_{z} \\
        \end{bmatrix}
\end{align*}
, \quad

\begin{align*}
  {\bf b} &= \begin{bmatrix}
          b_{x} \\
          b_{y} \\
          b_{z} \\
        \end{bmatrix}
\end{align*}
$$

$$
\theta = \cos^{-1}\biggl(\frac{{\bf a} \cdot {\bf b}}{|{\bf a}| |{\bf b}|}\biggr)
$$


### Exit velocity of a tennis ball in 2D
I've got a backyard tennis ball launcher where you can choose the launch angle and speed. Let's ignore the 3-dimensional aspect of the world right now and think in 2D. This results in a two dimensional input (launch angle and speed) and a two dimensional output for the velocity (x and y components). This **is** a vector valued function.

$$
\begin{align*}
  {\bf v} &= \begin{bmatrix}
          v_{x} \\
          v_{y} \\
        \end{bmatrix} &= \begin{bmatrix}
          v \cos(\theta) \\
          v \sin(\theta) \\
        \end{bmatrix}
\end{align*}
$$


### Force-displacement history for an automotive shock absorber
Let's say you're a suspension engineer for a car company. You are trying to model the displacement of the suspension system across a car's simulated route. Simplifying a lot of physics, we can obtain the displacement ($x$) of the shock absorber by knowing the force ($F$) acting on it and the spring constant ($k$): $x = F/k$. Given a recording of force measurements from 1000 timepoints in a lab test, we can compute the corresponding shock displacement at each point.

This results in 1000 inputs and 1000 outputs and this is a vector valued function.

## Brief math theory of derivative arrays (Jacobians)

I'll now detail some basic theory behind the derivatives of vector valued functions. Sec. 6.1 in [Engineering Design Optimization](https://mdobook.github.io/) also presents this information.

In the case of a function $f(x)$ where the input and output are both scalars, we get:

$$
\begin{equation}
\underset{1\times 1}{\frac{\partial f}{\partial x}}
\end{equation}
$$

In the general case we have an array called the *Jacobian* which contains the gradient information for vector valued functions. Its size is based on the number of inputs $n_x$ and the number of outputs $n_f$ as such:

$$
J_{f}=\frac{\partial f}{\partial x}=\left[\begin{array}{c}
\nabla f_{1}^\intercal \\
\vdots \\
\nabla f_{n_{f}}^\intercal
\end{array}\right]=\underbrace{\left[\begin{array}{ccc}
\frac{\partial f_{1}}{\partial x_{1}} & \cdots & \frac{\partial f_{1}}{\partial x_{n_{x}}} \\
\vdots & \ddots & \vdots \\
\frac{\partial f_{n_{f}}}{\partial x_{1}} & \cdots & \frac{\partial f_{n_{f}}}{\partial x_{n_{x}}}
\end{array}\right]}_{\left(n_{f} \times n_{x}\right)}
$$

Here is a reproduction of Ex. 6.1 from [Engineering Design Optimization](https://mdobook.github.io/) to show how to obtain the derivatives of a simple vector valued function.

$$
f(x)=\left[\begin{array}{l}
f_{1}\left(x_{1}, x_{2}\right) \\
f_{2}\left(x_{1}, x_{2}\right)
\end{array}\right]=\left[\begin{array}{c}
x_{1} x_{2}+\sin x_{1} \\
x_{1} x_{2}+x_{2}^{2}
\end{array}\right]
$$

Differentiating symbolically, we get:

$$
\frac{\partial f}{\partial x}=\left[\begin{array}{cc}
x_{2}+\cos x_{1} & x_{1} \\
x_{2} & x_{1}+2 x_{2}
\end{array}\right]
$$

Evaluating this at $x=(\pi/4, 2)$ yields

$$
\frac{\partial f}{\partial x}=\left[\begin{array}{ll}
2.707 & 0.785 \\
2.000 & 4.785
\end{array}\right]
$$

You can think of computing the Jacobian entries as individually computing derivatives of scalars with respect to scalars and putting them in an array. But I would not suggest thinking that way, especially in more complex cases. It's helpful to know that deep down all vector valued functions are just scalar functions smashed together. Often times, though, there will be patterns in the derivatives that you should harness when computing Jacobians for vector valued functions.

## Example case in OpenMDAO

Here's a simple example case implemented in OpenMDAO where we have three inputs and two outputs. We sum each neighboring value in the input vector to obtain the output vector.

In [13]:
import openmdao.api as om


class SystemOfEquations(om.ExplicitComponent):
    def setup(self):
        self.add_input('x', shape=3)
        self.add_output('f', shape=2)

    def compute(self, inputs, outputs):
        outputs['f'][0] = inputs['x'][0] + inputs['x'][1]
        outputs['f'][1] = inputs['x'][1] + inputs['x'][2]


prob = om.Problem()
prob.model.add_subsystem('subsystem', SystemOfEquations(), promotes=['*'])

prob.setup()
prob.set_val('x', [2., 5., 10.])

prob.run_model()
print(prob['f'])

[ 7. 15.]


Okay that's great, but we haven't implemented the derivatives for this component. Let's do that now. It's sort of a ridiculously simple case but I'm doing it on purpose so that we are not distracted by the complexity of the derivatives.

We rewrite the previous component and add analytic derivatives in the `compute_partials` method. Then, we check those derivatives against a finite difference approximation using the `check_partials` method.

In [14]:
class SystemOfEquations(om.ExplicitComponent):
    def setup(self):
        self.add_input('x', shape=3)
        self.add_output('f', shape=2)
        self.declare_partials(of='f', wrt='x')

    def compute(self, inputs, outputs):
        outputs['f'][0] = inputs['x'][0] + inputs['x'][1]
        outputs['f'][1] = inputs['x'][1] + inputs['x'][2]

    def compute_partials(self, inputs, partials):
        partials['f', 'x'][0, 0] = 1.
        partials['f', 'x'][0, 1] = 1.
        partials['f', 'x'][1, 1] = 1.
        partials['f', 'x'][1, 2] = 1.

prob = om.Problem()
prob.model.add_subsystem('subsystem', SystemOfEquations(), promotes=['*'])

prob.setup()
prob.set_val('x', [2., 5., 10.])

prob.run_model()
prob.check_partials(compact_print=True);

----------------------------------------
Component: SystemOfEquations 'subsystem'
----------------------------------------

'<output>' wrt '<variable>' | calc mag.  | check mag. | a(cal-chk) | r(cal-chk)
-------------------------------------------------------------------------------

'f'        wrt 'x'          | 2.0000e+00 | 2.0000e+00 | 1.2868e-09 | 6.4340e-10

#######################################################################
Sub Jacobian with Largest Relative Error: SystemOfEquations 'subsystem'
#######################################################################

'<output>' wrt '<variable>' | calc mag.  | check mag. | a(cal-chk) | r(cal-chk)
-------------------------------------------------------------------------------
'f'        wrt 'x'          | 2.0000e+00 | 2.0000e+00 | 1.2868e-09 | 6.4340e-10


Now of course, that was a trivially simple system. But I want to highlight some improvements to the code.

The first is that because all the derivative values are constant and do not depend on the input values, we can simply declare them in the `setup` method instead of using the `compute_partials` method. This will instead tell OpenMDAO the derivative values and that they never change. This is only possible for certain types of functions but saves computational cost by avoiding running computations in `compute_partials` unnecessarily.

Let's give this a naive try below.

In [15]:
class SystemOfEquations(om.ExplicitComponent):
    def setup(self):
        self.add_input('x', shape=3)
        self.add_output('f', shape=2)
        self.declare_partials(of='f', wrt='x', val=1.)

    def compute(self, inputs, outputs):
        outputs['f'][0] = inputs['x'][0] + inputs['x'][1]
        outputs['f'][1] = inputs['x'][1] + inputs['x'][2]

prob = om.Problem()
prob.model.add_subsystem('subsystem', SystemOfEquations(), promotes=['*'])

prob.setup()
prob.set_val('x', [2., 5., 10.])

prob.run_model()
prob.check_partials(compact_print=True);

----------------------------------------
Component: SystemOfEquations 'subsystem'
----------------------------------------

'<output>' wrt '<variable>' | calc mag.  | check mag. | a(cal-chk) | r(cal-chk)
-------------------------------------------------------------------------------

'f'        wrt 'x'          | 2.4495e+00 | 2.0000e+00 | 1.4142e+00 | 7.0711e-01 >ABS_TOL >REL_TOL

#######################################################################
Sub Jacobian with Largest Relative Error: SystemOfEquations 'subsystem'
#######################################################################

'<output>' wrt '<variable>' | calc mag.  | check mag. | a(cal-chk) | r(cal-chk)
-------------------------------------------------------------------------------
'f'        wrt 'x'          | 2.4495e+00 | 2.0000e+00 | 1.4142e+00 | 7.0711e-01


The derivative check didn't pass! Maybe it's obvious to you or maybe it's not clear why this is the case. After all, each of the derivative values we computed were 1, and then we said the Jacobian entry values are 1. However, only *some* of them are 1. I've fallen victim to thinking about this incorrectly before. We need to tell OpenMDAO *which* values are 1. Here's how we can do that.

In [16]:
import numpy as np


class SystemOfEquations(om.ExplicitComponent):
    def setup(self):
        self.add_input('x', shape=3)
        self.add_output('f', shape=2)

        jac = np.array([[1., 1., 0.],
                        [0., 1., 1.]])
        self.declare_partials(of='f', wrt='x', val=jac)

    def compute(self, inputs, outputs):
        outputs['f'][0] = inputs['x'][0] + inputs['x'][1]
        outputs['f'][1] = inputs['x'][1] + inputs['x'][2]

prob = om.Problem()
prob.model.add_subsystem('subsystem', SystemOfEquations(), promotes=['*'])

prob.setup()
prob.set_val('x', [2., 5., 10.])

prob.run_model()
prob.check_partials(compact_print=True);

----------------------------------------
Component: SystemOfEquations 'subsystem'
----------------------------------------

'<output>' wrt '<variable>' | calc mag.  | check mag. | a(cal-chk) | r(cal-chk)
-------------------------------------------------------------------------------

'f'        wrt 'x'          | 2.0000e+00 | 2.0000e+00 | 1.2868e-09 | 6.4340e-10

#######################################################################
Sub Jacobian with Largest Relative Error: SystemOfEquations 'subsystem'
#######################################################################

'<output>' wrt '<variable>' | calc mag.  | check mag. | a(cal-chk) | r(cal-chk)
-------------------------------------------------------------------------------
'f'        wrt 'x'          | 2.0000e+00 | 2.0000e+00 | 1.2868e-09 | 6.4340e-10


Fantastic, we did it better! But what if our component is dealing with bigger input and output dimensionality? Let's generalize what we've done and make it a little slicker.

In [17]:
class SystemOfEquations(om.ExplicitComponent):
    def setup(self):
        # Set the size of the input and output
        n = 100
        self.add_input('x', shape=n+1)
        self.add_output('f', shape=n)

        # Construct an array and fill it with the Jacobian info
        jac = np.zeros((n, n+1))
        arange = np.arange(n)
        jac[arange, arange] = 1.
        jac[arange, arange+1] = 1.
        
        # Give this Jacobian to OpenMDAO
        self.declare_partials(of='f', wrt='x', val=jac)

    def compute(self, inputs, outputs):
        # Loop through all the indices and add up the two neighboring
        # inpupt values to obtain the output values
        for idx, f in enumerate(outputs['f']):
            outputs['f'][idx] = inputs['x'][idx] + inputs['x'][idx+1]

prob = om.Problem()
prob.model.add_subsystem('subsystem', SystemOfEquations(), promotes=['*'])

prob.setup()

prob.run_model()
prob.check_partials(compact_print=True);

----------------------------------------
Component: SystemOfEquations 'subsystem'
----------------------------------------

'<output>' wrt '<variable>' | calc mag.  | check mag. | a(cal-chk) | r(cal-chk)
-------------------------------------------------------------------------------

'f'        wrt 'x'          | 1.4142e+01 | 1.4142e+01 | 1.9768e-09 | 1.3978e-10

#######################################################################
Sub Jacobian with Largest Relative Error: SystemOfEquations 'subsystem'
#######################################################################

'<output>' wrt '<variable>' | calc mag.  | check mag. | a(cal-chk) | r(cal-chk)
-------------------------------------------------------------------------------
'f'        wrt 'x'          | 1.4142e+01 | 1.4142e+01 | 1.9768e-09 | 1.3978e-10


Cool! Now we have a system and its corresponding derivatives set up in a very general and pretty efficient way. The number of lines of code we needed to use to handle a bigger system wasn't that bad. Of course there are some improvements we can make, but for now that's pretty good.

Zooming out from this simple case, there are many intricacies and subtlties to obtaining derivatives of vector valued functions. Depending on your model and problems, they may or may not be worth considering. We'll detail a few of them in the next few sections.

## Sparsity in the Jacobian can be exploited

In the immediately prior example we had a 100x101 Jacobian but only 200 non-zero entries. That's about 2% of the array occupied by numbers! There's 98% of that array just sitting these doing nothing! We can take advantage of that sparsity.

For large problems we want to avoid instantiating Jacobian arrays in a "dense" fashion if they are quite sparse. In our prior example, we had a dense and largely empty array. Instead, we should represent our Jacobian in a "sparse" way by only telling OpenMDAO about the non-zero entries.

Problems with sparse gradient arrays are quite common in engineering. Here are a few cases:
- The flow properties within a CFD cell only depend directly on the nearby geometry.
- For an aircraft trajectory the lift and drag at a quasi-steady point are only dependent on the current flight conditions. No other points in the time history affect that performance.
- The support structure for a small scale wind turbine consists of many small steel beams welded together. There is inherent sparsity in the gradients for this structure, e.g. increasing the thickness of a beam at the base of the support does not change the properties of a beam at the top of the tower.

Let's revisit the example case from above and take advantage of the sparsity inherent in this problem using [OpenMDAO's sparsity support.](https://openmdao.org/newdocs/versions/latest/features/core_features/working_with_derivatives/sparse_partials.html)

In [18]:
class SystemOfEquations(om.ExplicitComponent):
    def setup(self):
        # Set the size of the input and output
        n = 100
        self.add_input('x', shape=n+1)
        self.add_output('f', shape=n)

        # Construct arrays for the indices for the sparse array.
        # Through this, OpenMDAO never creates a dense array for the Jacobian
        # and instead uses a sparse array representation.
        arange = np.arange(n)
        rows = np.hstack((arange, arange))
        cols = np.hstack((arange, arange+1))
        
        # Give this Jacobian to OpenMDAO
        self.declare_partials(of='f', wrt='x', val=1., rows=rows, cols=cols)

    def compute(self, inputs, outputs):
        # Loop through all the indices and add up the two neighboring
        # inpupt values to obtain the output values
        for idx, f in enumerate(outputs['f']):
            outputs['f'][idx] = inputs['x'][idx] + inputs['x'][idx+1]

prob = om.Problem()
prob.model.add_subsystem('subsystem', SystemOfEquations(), promotes=['*'])

prob.setup()

prob.run_model()
prob.check_partials(compact_print=True);

----------------------------------------
Component: SystemOfEquations 'subsystem'
----------------------------------------

'<output>' wrt '<variable>' | calc mag.  | check mag. | a(cal-chk) | r(cal-chk)
-------------------------------------------------------------------------------

'f'        wrt 'x'          | 1.4142e+01 | 1.4142e+01 | 1.9768e-09 | 1.3978e-10

#######################################################################
Sub Jacobian with Largest Relative Error: SystemOfEquations 'subsystem'
#######################################################################

'<output>' wrt '<variable>' | calc mag.  | check mag. | a(cal-chk) | r(cal-chk)
-------------------------------------------------------------------------------
'f'        wrt 'x'          | 1.4142e+01 | 1.4142e+01 | 1.9768e-09 | 1.3978e-10


Sparse derivatives are not always fixed values. In general, you can declare the sparsity of the Jacobian in your `setup` method and then fill in the actual values of those non-zero Jacobian entries in the `compute_partials` method.

It's easy to encounter models with sparsity patterns that are much more complex than what I've shown in the examples. Sometimes it's challenging to even recognize the sparsity pattern, let alone put it into code and compute the derivatives. Luckily, OpenMDAO has some helpful tools built in that can reduce your effort.

One of those helpful tools is the `view_coloring` command-line tool, shown on [this doc page](https://openmdao.org/newdocs/versions/latest/features/core_features/working_with_derivatives/simul_derivs.html). Although it's called `view_coloring`, it also helps you see the sparsity pattern for your components or systems, as shown in the example image below. The next section explains coloring in more detail.

I find this visual to be quite helpful when I'm trying to figure out the sparsity pattern for my systems, especially because it's challenging to see patterns when looking at text or math but it's much easier to see those patterns visually.

![View coloring results](coloring_viewer.png)

## Derivative coloring can also help

"Coloring" is the idea that multiple derivative values can be computed simultaneously because those derivatives have no effect on each other. The name comes from the idea of coloring the Jacobian based on the values that are independent of each other, using the minimum number of colors possible. Think of coloring the countries on a map where any bordering country has a different color from its neighbors.

Coloring decreases the computational cost associated with computing derivatives when your model has separable functions of interest. [This doc page](http://openmdao.org/twodocs/versions/latest/theory_manual/advanced_linear_solvers_special_cases/separable.html) goes into much more detail and has some fantastic visuals.

Please see other lessons for a more detailed examination of coloring, specifically [[Partial derivative coloring]] and [[Total derivative coloring]].

For the purposes of this lesson, know that coloring can help decrease the cost of computing gradients for vector valued functions. If your model is too expensive during derivative computation, you can look into using coloring to decrease the time to solve the linear system.

[This doc page](https://openmdao.org/newdocs/versions/latest/features/core_features/working_with_derivatives/simul_derivs.html) is helpful for more info on how to look at coloring within problems in OpenMDAO and [this page](https://openmdao.org/newdocs/versions/latest/features/experimental/approx_coloring.html) is helpful if you are approximating your derivatives, e.g. using finite difference or complex step. [This last page on setting up linear solvers](https://openmdao.org/newdocs/versions/latest/theory_manual/setup_linear_solvers.html) is also relevant for some cases.

## Other resources

This lesson just introduces and scratches the surface of getting derivatives of vector valued functions. Other lessons will go into detail about more advanced techniques and performance improvements.